In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as mse

In [2]:
def distance(x1, y1, x2, y2):
    return np.sqrt((x1 - x2)**2 + (y1 - y2)**2)

In [17]:
df_path = "./tobii_test_updv2_mixed_face_screen_feat_balanced_pred.csv"
df_path_golden = "./golden_test.csv"
user_specs_path = "../data/participant_characteristics.csv"
user_specs = pd.read_csv(user_specs_path)[[
    "Display Height (pixels)", "Display Width (pixels)", 'Participant ID',
    'Screen Width (cm)', 'Screen Height (cm)', 'Distance From Screen (cm)',
    'Self-Reported Vision'
]]
df_pred = pd.read_csv(df_path)
df_pred["distances"] = df_pred[['x_normalized', 'y_normalized', 'pred_x', 'pred_y']].apply(lambda x: distance(*x), axis = 1)
df_pred["user_id"] = df_pred["full_name"].apply(lambda x: x.split("#")[0])
df_pred = df_pred.merge(user_specs, left_on = "user_id", right_on = 'Participant ID', how = "inner", suffixes=["", "_"]).drop(columns = ['Participant ID'])

df_golden = pd.read_csv(df_path_golden)
df_pred_golden = df_golden.merge(df_pred, on = "paths", how = "inner", suffixes=["", "_"])

In [18]:
df_pred.head()

,paths,x_normalized,y_normalized,full_name,user_id,Screen Width (cm),Screen Height (cm),Distance From Screen (cm),pred_x,pred_y,distances,Display Height (pixels),Display Width (pixels),Screen Width (cm)_,Screen Height (cm)_,Distance From Screen (cm)_,Self-Reported Vision
0,/home/ubuntu/projects/tweakle/data/tobii_image...,0.194715,0.582539,P_38#1492618354027_23_-study-tooth_abscess_wri...,P_38,33.17,20.73,50.0,0.234828,0.533801,0.063123,900,1440,33.17,20.73,50.0,Normal
1,/home/ubuntu/projects/tweakle/data/tobii_image...,0.200532,0.493025,P_38#1492618354027_16_-study-educational_advan...,P_38,33.17,20.73,50.0,0.276412,0.409475,0.112864,900,1440,33.17,20.73,50.0,Normal
2,/home/ubuntu/projects/tweakle/data/tobii_image...,0.247674,0.453015,P_38#1492618354027_24_-study-dot_test_final_in...,P_38,33.17,20.73,50.0,0.232039,0.495984,0.045725,900,1440,33.17,20.73,50.0,Normal
3,/home/ubuntu/projects/tweakle/data/tobii_image...,0.217074,0.442437,P_38#1492618354027_7_-study-benefits_of_runnin...,P_38,33.17,20.73,50.0,0.249681,0.423664,0.037625,900,1440,33.17,20.73,50.0,Normal
4,/home/ubuntu/projects/tweakle/data/tobii_image...,0.123454,0.445198,P_38#1492618354027_9_-study-benefits_of_runnin...,P_38,33.17,20.73,50.0,0.146342,0.461879,0.028321,900,1440,33.17,20.73,50.0,Normal


In [19]:
df_pred["Self-Reported Vision"].value_counts()

Self-Reported Vision
Normal      8078
Glasses     4426
Contacts    1496
Name: count, dtype: int64

In [20]:
def average_angle(df):
    df_ = df.copy()
    df_["x"] = df_["x_normalized"] * df_["Screen Width (cm)"]
    df_["pred_x_"] = df_["pred_x"] * df_["Screen Width (cm)"]
    df_["y"] = df_["y_normalized"] * df_["Screen Height (cm)"]
    df_["pred_y_"] = df_["pred_y"] * df_["Screen Height (cm)"]
    df_["center_y"] = df_["Screen Height (cm)"] * 0.5
    df_["center_x"] = df_["Screen Width (cm)"] * 0.5
    df_["distances_center_pred"] = df_[['center_x', 'center_y', 'pred_x_', 'pred_y_']].apply(lambda x: distance(*x), axis = 1)
    df_["distances_center_gt"] = df_[['center_x', 'center_y', 'x', 'y']].apply(lambda x: distance(*x), axis = 1)
    df_["tg_center_gt"] = df_["distances_center_gt"] / df_["Distance From Screen (cm)"]
    df_["tg_center_pred"] = df_["distances_center_pred"] / df_["Distance From Screen (cm)"]
    df_["angle_gt"] = np.arctan(df_["tg_center_gt"].values) * 180 / np.pi
    df_["angle_pred"] = np.arctan(df_["tg_center_pred"].values) * 180 / np.pi
    angles = (df_["angle_gt"] - df_["angle_pred"]).abs()
    return angles.mean(), angles.std()

In [21]:
def average_pixel_distance(df):
    df_ = df.copy()
    df_["x"] = df_["x_normalized"] * df_["Display Width (pixels)"]
    df_["pred_x_"] = df_["pred_x"] * df_["Display Width (pixels)"]
    df_["y"] = df_["y_normalized"] * df_["Display Height (pixels)"]
    df_["pred_y_"] = df_["pred_y"] * df_["Display Height (pixels)"]
    df_["distances"] = df_[['x', 'y', 'pred_x_', 'pred_y_']].apply(lambda x: distance(*x), axis = 1)
    return df_["distances"].mean(), df_["distances"].std()

In [22]:
def compute_metrics(df):
    preds = df[["pred_x", "pred_y"]].values
    labels = df[["x_normalized", "y_normalized"]].values
    
    mape_value = mape(labels, preds)
    print(f"Test MAPE: {round(mape_value, 4)}")
    
    rmse_value = mse(labels, preds, squared=True)
    print(f"Test RMSE: {round(rmse_value, 4)}")
    
    av_rel_dist = df["distances"].mean()
    print(f"Average relative distance: {round(av_rel_dist, 4)} +- {round(df['distances'].std(), 3)}")

    av_pixel_dist, std = average_pixel_distance(df)
    print(f"Average pixel distance: {round(av_pixel_dist, 1)} +- {round(std, 1)}")
    av_angle, std = average_angle(df)
    print(f"Average angle: {round(av_angle, 2)} +- {round(std, 2)}")

All test

In [23]:
compute_metrics(df_pred)

Test MAPE: 0.4502
Test RMSE: 0.0138
Average relative distance: 0.1342 +- 0.098
Average pixel distance: 190.7 +- 136.6
Average angle: 2.62 +- 2.25


Glasses

In [24]:
df_glasses = df_pred[df_pred['Self-Reported Vision'] == "Glasses"]

In [25]:
compute_metrics(df_glasses)

Test MAPE: 0.5181
Test RMSE: 0.0153
Average relative distance: 0.1427 +- 0.101
Average pixel distance: 202.6 +- 141.5
Average angle: 2.78 +- 2.39


Normal

In [26]:
df_glasses = df_pred[df_pred['Self-Reported Vision'] != "Glasses"]

In [27]:
compute_metrics(df_glasses)

Test MAPE: 0.4189
Test RMSE: 0.0131
Average relative distance: 0.1303 +- 0.096
Average pixel distance: 185.1 +- 133.9
Average angle: 2.55 +- 2.18


Golden test

In [28]:
compute_metrics(df_pred_golden)

Test MAPE: 1.3518
Test RMSE: 0.0167
Average relative distance: 0.1535 +- 0.114
Average pixel distance: 190.8 +- 111.1
Average angle: 3.11 +- 2.56


In [29]:
compute_metrics(df_pred_golden)

Test MAPE: 1.3518
Test RMSE: 0.0167
Average relative distance: 0.1535 +- 0.114
Average pixel distance: 190.8 +- 111.1
Average angle: 3.11 +- 2.56
